In [ ]:
import torch
from torch import nn
import os

####Setting up device-agnostic code

In [ ]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
device

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive',force_remount=True)

In [ ]:
#Posixpath module in Python- functions for manipulating file paths in a portable manner
#Posixpath is a child node of Path()
from pathlib import Path

In [ ]:
#setting up training and validation data paths
data_path = Path("/content/gdrive/MyDrive/")
image_path = data_path / "CottonData-StandardFormat"

### 0. Dataset Visualization

In [ ]:
from PIL import Image
import numpy as np
import os

def create_image_collage(image_folder, collage_size, output_path):
    # Get all image files in the specified folder
    #image_files = [f for f in os.listdir(image_folder) if f.endswith(('png', 'jpg', 'jpeg', 'gif'))]

    # Select a fixed number of images (e.g., 100)
    num_images_to_select = 100
    image_files = [f for f in os.listdir(image_folder) if f.endswith(('png', 'jpg', 'jpeg', 'gif'))][:num_images_to_select]

    # Shuffle the image files for randomness
    np.random.shuffle(image_files)

    # Open and resize images to fit the collage
    images = [Image.open(os.path.join(image_folder, img)).resize(collage_size) for img in image_files]

    # Calculate the dimensions of the collage
    collage_width = collage_size[0] * int(np.ceil(np.sqrt(len(images))))
    collage_height = collage_size[1] * int(np.ceil(len(images) / np.sqrt(len(images))))

    # Create a blank canvas for the collage
    collage = Image.new('RGB', (collage_width, collage_height), (255, 255, 255))

    # Paste each image onto the canvas
    x_offset, y_offset = 0, 0
    for img in images:
        collage.paste(img, (x_offset, y_offset))
        x_offset += collage_size[0]
        if x_offset >= collage_width:
            x_offset = 0
            y_offset += collage_size[1]

    # Save the collage to the specified output path
    collage.save(output_path)

# Example usage:
image_folder_path = "path_of_validation_dataset_of_crop"
collage_size = (100, 100)  # Adjust the size of each image tile
output_collage_path = "path_of_output_tiled_dataset_of_crop.jpg"

create_image_collage(image_folder_path, collage_size, output_collage_path)

In [ ]:
from PIL import Image
import numpy as np
import os

def create_image_collage(image_folder, collage_size, output_path):
    # Get all image files in the specified folder
    #image_files = [f for f in os.listdir(image_folder) if f.endswith(('png', 'jpg', 'jpeg', 'gif'))]

    # Select a fixed number of images (e.g., 100)
    num_images_to_select = 100
    image_files = [f for f in os.listdir(image_folder) if f.endswith(('png', 'jpg', 'jpeg', 'gif'))][:num_images_to_select]


    # Shuffle the image files for randomness
    np.random.shuffle(image_files)

    # Open and resize images to fit the collage
    images = [Image.open(os.path.join(image_folder, img)).resize(collage_size) for img in image_files]

    # Calculate the dimensions of the collage
    collage_width = collage_size[0] * int(np.ceil(np.sqrt(len(images))))
    collage_height = collage_size[1] * int(np.ceil(len(images) / np.sqrt(len(images))))

    # Create a blank canvas for the collage
    collage = Image.new('RGB', (collage_width, collage_height), (255, 255, 255))

    # Paste each image onto the canvas
    x_offset, y_offset = 0, 0
    for img in images:
        collage.paste(img, (x_offset, y_offset))
        x_offset += collage_size[0]
        if x_offset >= collage_width:
            x_offset = 0
            y_offset += collage_size[1]

    # Save the collage to the specified output path
    collage.save(output_path)

# Example usage:
image_folder_path = "path_of_validation_dataset_of_weed"
collage_size = (100,100)  # Adjust the size of each image tile
output_collage_path = "path_of_output_tiled_dataset_of_weed.jpg"

create_image_collage(image_folder_path, collage_size, output_collage_path)

### 1. Data preparation

In [ ]:
def walk_through_dir(dir_path):
  for dirpath, dirnames, filenames in os.walk(dir_path):
    print(f"there are {len(dirnames)} directories and {len(filenames)} images in {dirpath}.")


In [ ]:
walk_through_dir(image_path)

In [ ]:
train_dir = image_path / "Train"
val_dir = image_path / "Validation"
test_dir = image_path/ "Test"
train_dir, val_dir, test_dir

In [ ]:
os.listdir(image_path)

### 1.1 Visualizing a single image

In [ ]:
import random
from PIL import Image

#set seed
random.seed(42)

# 1. get all image paths (* means any combination)
image_path_list = list(image_path.glob("*/*/*.jpg"))

# 2. get random image path
random_img_path = random.choice(image_path_list)

# 3. get image class from path name
image_class = random_img_path.parent.stem

# 4. open image
img = Image.open(random_img_path)

# 5. print metadata
print(f"Random image path: {random_img_path}")
print(f"Image class: {image_class}")
print(f"Image height: {img.height}")
print(f"Image width: {img.width}")
img

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
# visualizing with matplotlib
# convert the img to numpy for visualizing in matplotlib

# turn the img in an array
img_as_array = np.asarray(img)

# plot img with matplotlib
plt.figure(figsize=(6,6)) #6inch by 6inch
plt.imshow(img_as_array)
plt.title(f"Image class: {image_class}, Image shape: {img_as_array.shape} -> [H, W, CC]")
plt.axis(False);

### 1.2 Transforming data
Before we can use our image data with PyTorch we need to:

1.Turn it into tensors (numerical representations of our images).

2.Turn it into a torch.utils.data.Dataset and subsequently a torch.utils.data.DataLoader, we'll call these Dataset and DataLoader for short.

In [ ]:
import torch
from torch.utils.data import DataLoader
from torchvision import datasets, transforms

1.Resize the images using transforms.Resize() (from about 512x512 to 128X128, the same shape as the images on the CNN Explainer website).

2.Flip our images randomly on the horizontal using transforms.RandomHorizontalFlip() (this could be considered a form of data augmentation because it will artificially change our image data).

3.Turn our images from a PIL image to a PyTorch tensor using transforms.ToTensor().

In [ ]:
# write transform for image
data_transform = transforms.Compose([
    # Resize the image to 128x128
    transforms.Resize(size=(128,128), antialias=None),
    # flip the images randomly on the horizontal
    transforms.RandomHorizontalFlip(p=0.5), #p=prob of flip
    # turn the image into a tensor
    transforms.ToTensor(),# this also converts all pixel values from 0 to 255 to be between 0.0 and 1.0
    #transforms.Normalize(mean=[0.485,0.456,0.406], std=[0.229,0.224,0.225])
])

In [ ]:
# applying the above transforms on various images.
def plot_transformed_images(image_path, transform, n=3, seed=42):

  """Plots a series of random images from image_paths.

    Will open n image paths from image_paths, transform them
    with transform and plot them side by side.

    Args:
        image_paths (list): List of target image paths.
        transform (PyTorch Transforms): Transforms to apply to images.
        n (int, optional): Number of images to plot. Defaults to 3.
        seed (int, optional): Random seed for the random generator. Defaults to 42.
    """
  #random.seed(seed)
  random_image_paths = random.sample(image_path, k=n)
  for image_path in random_image_paths:
    with Image.open(image_path) as f:
      fig, ax = plt.subplots(1,2)
      ax[0].imshow(f)
      ax[0].set_title(f"original \nsize: {f.size}")
      ax[0].axis("off")

      # Transform and plot image
      # Note: permute() will change shape of image to suit matplotlib
      # (PyTorch default is [C, H, W] but Matplotlib is [H, W, C])
      transformed_image = transform(f).permute(1,2,0)
      ax[1].imshow(transformed_image)
      ax[1].set_title(f"Tranformed \nsize: {transformed_image.shape}")
      ax[1].axis("off")

      fig.suptitle(f"class: {image_path.parent.stem}", fontsize=16)
plot_transformed_images(image_path_list,
                        transform=data_transform,
                        n=3)


### 2. Loading image data using ImageFolder
Alright, time to turn our image data into a Dataset capable of being used with PyTorch.

In [ ]:
# use ImageFolder to create dataset
from torchvision import datasets
from torchvision.datasets import ImageFolder
train_data = datasets.ImageFolder(root=train_dir,
                                  transform=data_transform,
                                  target_transform=None)
val_data = datasets.ImageFolder(root=val_dir,
                                transform=data_transform)
print(f"Train data :\n{train_data}\nVal data :\n{val_data}")

In [ ]:
test_data = datasets.ImageFolder(root=test_dir,
                                 transform=data_transform)
print(f"Test data: \n{test_data}")

In [ ]:
# get class names as a list
class_names = train_data.classes
class_names

In [ ]:
# can also get class names as a dict
class_dict = train_data.class_to_idx
class_dict

In [ ]:
# check the length
len(train_data), len(val_data)

In [ ]:
img, label = train_data [0][0], train_data[0][1]
print(f"image tensor: {img}")
print(f"image shape: {img.shape}")
print(f"image datatype: {img.dtype}")
print(f"image label: {label}")
print(f"label datatype: {type(label)}")

In [ ]:
# Rearrange the order of dimensions
img_permute = img.permute(1, 2, 0)

# Print out different shapes (before and after permute)
print(f"Original shape: {img.shape} -> [color_channels, height, width]")
print(f"Image permute shape: {img_permute.shape} -> [height, width, color_channels]")

# Plot the image
plt.figure(figsize=(10, 7))
plt.imshow(img.permute(1, 2, 0))
plt.axis("off")
plt.title(class_names[label], fontsize=14);

### 3.Turn loaded images into DataLoader's
Turning our Dataset's into DataLoader's makes them iterable so a model can go through learn the relationships between samples and targets (features and labels).

In [ ]:
# Turn train and val Datasets into DataLoaders
from torch.utils.data import DataLoader
BATCH_SIZE = 32
NUM_WORKERS = os.cpu_count()
train_dataloader = DataLoader(dataset=train_data,
                              batch_size=BATCH_SIZE, # how many samples per batch?
                              num_workers=NUM_WORKERS, # how many subprocesses to use for data loading? (higher = more)
                              shuffle=True) # shuffle the data?

val_dataloader = DataLoader(dataset=val_data,
                             batch_size=BATCH_SIZE,
                             num_workers=NUM_WORKERS,
                             shuffle=False) # don't usually need to shuffle testing data

train_dataloader, val_dataloader

In [ ]:
test_dataloader = DataLoader(dataset=test_data,
                             batch_size=BATCH_SIZE,
                             num_workers=NUM_WORKERS,
                             shuffle=False)
test_dataloader

In [ ]:
img, label = next(iter(train_dataloader))

# Batch size will now be 1, try changing the batch_size parameter above and see what happens
print(f"Image shape: {img.shape} -> [batch_size, color_channels, height, width]")
print(f"Label shape: {label.shape}")

### Other forms of transforms (Data Augmentation)
Data augmentation is the process of altering your data in such a way that you artificially increase the diversity of your training set.

You usually don't perform data augmentation on the test set. The idea of data augmentation is to to artificially increase the diversity of the training set to better predict on the testing set.

In [ ]:
from torchvision import transforms

train_transforms = transforms.Compose([
    transforms.Resize((224,224)),
    transforms.TrivialAugmentWide(num_magnitude_bins=31),
    transforms.ToTensor()
])
# Don't need to perform augmentation on the val data
val_transforms = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor()
])

In [ ]:
# Get all image paths
image_path_list = list(image_path.glob("*/*/*.jpg"))

#plot random image
plot_transformed_images(
    image_path=image_path_list,
    transform=train_transforms,
    n=5,
    seed=None
)

### Creating transforms and loading data for Model 0

we will use earlier transforms and not create a new transform.Also using previous dataloaders.

### 4.Create TinyVGG model class

In [ ]:
class TinyVGG(nn.Module):
  def __init__(self, input_shape: int, hidden_units: int, output_shape: int) -> None:
    super().__init__()
    self.conv_block_1 = nn.Sequential(
        nn.Conv2d(in_channels=input_shape,
                  out_channels=hidden_units,
                  kernel_size=3,
                  stride=1,
                  padding=1),
        nn.ReLU(),
        nn.Conv2d(in_channels=hidden_units,
                  out_channels=hidden_units,
                  kernel_size=3,
                  stride=1,
                  padding=1),
        nn.ReLU(),
        nn.MaxPool2d(kernel_size=2,
                     stride=2)
    )
    self.conv_block_2 = nn.Sequential(
        nn.Conv2d(hidden_units, hidden_units, kernel_size=3, padding=1),
        nn.ReLU(),
        nn.Conv2d(hidden_units, hidden_units, kernel_size=3, padding=1),
        nn.ReLU(),
        nn.MaxPool2d(2)
    )
    self.classifier = nn.Sequential(
        nn.Flatten(),
        nn.Linear(in_features = hidden_units*32*32,
                  out_features=output_shape)
    )
  def forward(self, x:torch.Tensor):
    x = self.conv_block_1(x)
    x = self.conv_block_2(x)
    x = self.classifier(x)
    return x

torch.manual_seed(42)
model_BCE_20epochs = TinyVGG(input_shape=3,
                  hidden_units=10,
                  output_shape=len(train_data.classes)).to(device)
model_BCE_20epochs

### 4.1 Try a forward pass on a single image(to test the model)

In [ ]:
# 1. Get a batch of images and labels from the DataLoader
img_batch, label_batch = next(iter(train_dataloader))

# 2. Get a single image from the batch and
#unsqueeze the image so its shape fits the model
img_single, label_single = img_batch[0].unsqueeze(dim=0), label_batch[0]
print(f"single image shape: {img_single.shape}")

# 3. Perform a forward pass on the single image
model_BCE_20epochs.eval()
with torch.inference_mode():
  pred = model_BCE_20epochs(img_single.to(device))

# 4. Print what's happening
# convert model logits -> pred probs -> pred labels
print(f"output logits: {pred}")
print(f"output prediction probabilities: {torch.sigmoid(pred)}")
print(f"output prediction label: {torch.round(torch.sigmoid(pred))}")
print(f"actual label: {label_single}")

### 5. Using torchinfo to get the idea of the shapes going through our model

In [ ]:
# Install torchinfo if it's not available, import it if it is
try:
    import torchinfo
except:
    !pip install torchinfo
    import torchinfo

from torchinfo import summary
summary(model_BCE_20epochs, input_size=[1, 3, 128, 128]) # do a test pass through of an example input size

### 6. Train and Test loop functions
Now let's make some training and test loop functions to train our model on the training data and evaluate our model on the validation data.

1. train_step() - takes in a model, a DataLoader, a loss function and an optimizer and trains the model on the DataLoader.

2. test_step() - takes in a model, a DataLoader and a loss function and evaluates the model on the DataLoader.

3. train() - performs 1. and 2. together for a given number of epochs and returns a results dictionary.

In [ ]:
def train_step(model: torch.nn.Module,
               dataloader: torch.utils.data.DataLoader,
               loss_fn: torch.nn.Module,
               optimizer: torch.optim.Optimizer):
  # Put model in train mode
  model.train()

  # Setup train loss and train accuracy values
  train_loss, train_acc = 0, 0

  # Loop through data loader data batches
  for batch, (X,y) in enumerate(dataloader):
    #send data to target device

    X,y = X.to(device), y.to(device)

    #print(f"y shape: {y.shape}")
    y = y.view(-1,1).float()
    #print(f"y shape after view: {y.shape}")

    # 1. Forward pass
    y_pred = model(X)
    #print(f"y_pred shape: {y_pred.shape}")

    # 2. Calculate and accumulate loss
    loss = loss_fn(y_pred, y)
    train_loss += loss.item()

    # 3. Optimizer zero grad
    optimizer.zero_grad()

    # 4. Loss backward
    loss.backward()

    # 5. Optimizer step
    optimizer.step()

    # Calculate and accumulate accuracy metric across all batches
    y_pred_class = torch.round(torch.sigmoid(y_pred))
    train_acc += (y_pred_class ==y).sum().item()/len(y_pred)

  # Adjust metrics to get average loss and accuracy per batch
  train_loss = train_loss/len(dataloader)
  train_acc = train_acc/len(dataloader)
  return train_loss, train_acc

In [ ]:
# The main difference here will be the test_step() won't take an optimizer
#and therefore won't perform gradient descent.
def val_step(model: torch.nn.Module,
              dataloader: torch.utils.data.DataLoader,
              loss_fn: torch.nn.Module):
    # Put model in eval mode
    model.eval()

    # Setup test loss and test accuracy values
    val_loss, val_acc = 0, 0

    # Turn on inference context manager
    with torch.inference_mode():
        # Loop through DataLoader batches
        for batch, (X, y) in enumerate(dataloader):
            # Send data to target device

            X, y = X.to(device), y.to(device)
            y = y.view(-1,1).float()

            # 1. Forward pass
            val_pred_logits = model(X)

            # 2. Calculate and accumulate loss
            loss = loss_fn(val_pred_logits, y)
            val_loss += loss.item()

            # Calculate and accumulate accuracy
            val_pred_labels = torch.round(torch.sigmoid(val_pred_logits))
            val_acc += ((val_pred_labels == y).sum().item()/len(val_pred_labels))

    # Adjust metrics to get average loss and accuracy per batch
    val_loss = val_loss / len(dataloader)
    val_acc = val_acc / len(dataloader)
    return val_loss, val_acc

### 7. Creating a train() function to combine train_step() and test_step()

Specificially, it'll:

1. Take in a model, a DataLoader for training and test sets, an optimizer, a loss function and how many epochs to perform each train and test step for.

2.  Create an empty results dictionary for train_loss, train_acc, test_loss and test_acc values (we can fill this up as training goes on).

3.Loop through the training and test step functions for a number of epochs.

4. Print out what's happening at the end of each epoch.

5. Update the empty results dictionary with the updated metrics each epoch.

6. Return the filled

In [ ]:
from tqdm.auto import tqdm

# 1. Take in various parameters required for training and test steps
def train(model: torch.nn.Module,
          train_dataloader: torch.utils.data.DataLoader,
          test_dataloader: torch.utils.data.DataLoader,
          optimizer: torch.optim.Optimizer,
          loss_fn: torch.nn.Module = nn.BCEWithLogitsLoss(),
          epochs: int = 5):
  # 2. Create empty results dictionary
  results = {"train_loss": [],
             "train_acc": [],
             "val_loss": [],
             "val_acc": []}

  # 3. Loop through training and validation steps for a number of epochs
  for epoch in tqdm(range(epochs)):
    train_loss, train_acc = train_step(model=model,
                                       dataloader=train_dataloader,
                                       loss_fn=loss_fn,
                                       optimizer=optimizer)
    val_loss, val_acc = val_step(model=model,
                                 dataloader=val_dataloader,
                                 loss_fn=loss_fn)
    # 4. Print out what is happening
    print(
        f"Epochs:{epoch+1}",
        f"train_loss: {train_loss:.4f}",
        f"train_acc: {train_acc:.4f}",
        f"val_loss: {val_loss:.4f}",
        f"val_acc: {val_acc:.4f}"
    )

    #  5. update dictionary
    results["train_loss"].append(train_loss)
    results["train_acc"].append(train_acc)
    results["val_loss"].append(val_loss)
    results["val_acc"].append(val_acc)

  # 6. return the filled results at the end of the epochs
  return results

### 7.1 Train and Evaluate model 0

In [ ]:
# Set random seeds
torch.manual_seed(42)
torch.cuda.manual_seed(42)

# Set number of epochs
NUM_EPOCHS = 20

# Recreate an instance of TinyVGG
model_BCE_20epochs = TinyVGG(input_shape=3, # number of color channels (3 for RGB)
                            hidden_units=10,
                            output_shape=1).to(device)

# Setup loss function and optimizer
loss_fn = nn.BCEWithLogitsLoss()
optimizer = torch.optim.SGD(params=model_BCE_20epochs.parameters(), lr=0.01)

# Start the timer
from timeit import default_timer as timer
start_time = timer()

# Train model_BCE_8epochs
model_BCE_20epochs_results = train(model=model_BCE_20epochs,
                                  train_dataloader=train_dataloader,
                                  test_dataloader=val_dataloader,
                                  optimizer=optimizer,
                                  loss_fn=loss_fn,
                                  epochs=NUM_EPOCHS)

# End the timer and print out how long it took
end_time = timer()
print(f"Total training time: {end_time-start_time:.3f} seconds")

In [ ]:
model_BCE_20epochs_results.keys()

### 8. Saving the Model

In [ ]:
from pathlib import Path

#create model directory path
MODEL_PATH = Path("path_of_CNN_model")
MODEL_PATH.mkdir(parents=True, #creating dir if it doesn't exist
                 exist_ok=True)

#create model save
MODEL_NAME = "CNN_crop_weed_data_model_BCE_20epochs.pth" #.pth/.pt for pytorch
MODEL_SAVE_PATH = MODEL_PATH / MODEL_NAME

#save the model state dict
#We'll call torch.save(obj, f)
#where obj is the target model's state_dict() and f is the filename of where to save the model.
print(f"Saving model to: {MODEL_SAVE_PATH}")
torch.save(obj=model_BCE_20epochs.state_dict(),
           f=MODEL_SAVE_PATH) #f is the file path i.e MODEL_SAVE_PATH

### 9. Loading a saved PyTorch model's state_dict()

Since we've now got a saved model state_dict() at models/01_pytorch_workflow_model_0.pth we can now load it in using torch.nn.Module.load_state_dict(torch.load(f)) where f is the filepath of our saved model state_dict().

Why call torch.load() inside torch.nn.Module.load_state_dict()?

Because we only saved the model's state_dict() which is a dictionary of learned parameters and not the entire model, we first have to load the state_dict() with torch.load() and then pass that state_dict() to a new instance of our model (which is a subclass of nn.Module).

In [ ]:
# create a new instance
# Instantiate a new instance of our model (this will be instantiated with random weights)
torch.manual_seed(42)

loaded_model_BCE_20epochs = TinyVGG(input_shape=3,
                                   hidden_units=10,
                                   output_shape=1).to(device)
# load in the save state_dict()
loaded_model_BCE_20epochs.load_state_dict(torch.load(f=MODEL_SAVE_PATH)) #f = file that we want to load

### 10. Plot the loss curves of Model 0
Loss curves show the model's results over time.

And they're a great way to see how your model performs on different datasets (e.g. training and val).

In [ ]:
def plot_loss_curves(results: dict[str, list[float]]):
  """Plots training curves of a result dictionary.
  """
  # Get the loss values of the results dictionary(training and val)
  loss = results['train_loss']
  val_loss = results['val_loss']

  # Get the accuracy values of the results dictionary(training and val)
  accuracy = results['train_acc']
  val_accuracy = results['val_acc']

  # Figure out how many epochs there were
  epochs = range(1,21,1) #range(len(results['train_loss']))

  # Setup a plot
  plt.figure(figsize=(14,8))
  x_ticks = np.arange(1, 21 , step=1)

  # Plot loss
  plt.plot(x_ticks, loss, label= 'train_loss')
  plt.plot(x_ticks, val_loss, label= 'val_loss')
  plt.title("Model Loss")
  plt.xlabel("Epochs")
  plt.ylabel("Loss")
  plt.xticks(x_ticks)
  plt.legend()

  # Plot accuracy
  plt.figure(figsize=(14,8))
  plt.plot(x_ticks, accuracy, label='train_accuracy')
  plt.plot(x_ticks, val_accuracy, label='val_accuracy')
  plt.title('Model Accuracy')
  plt.xlabel('Epochs')
  plt.ylabel('Accuracy')
  plt.xticks(x_ticks)
  plt.legend();

In [ ]:
plot_loss_curves(model_BCE_20epochs_results)

In [ ]:
import pandas as pd
model_BCE_20epochs_df = pd.DataFrame(model_BCE_20epochs_results)
model_BCE_20epochs_df

In [ ]:
from matplotlib import pyplot as plt
x_ticks = np.arange(0, 21 , step=1)
model_BCE_20epochs_df['val_acc'].plot(kind='line', figsize=(8, 4), title='val_acc')
plt.gca().spines[['top', 'right']].set_visible(False)
plt.xlabel("Epochs")
plt.xticks(x_ticks)
plt.ylabel("Accuracy")

In [ ]:
from matplotlib import pyplot as plt
x_ticks = np.arange(0, 21 , step=1)
model_BCE_20epochs_df['val_loss'].plot(kind='line', figsize=(8, 4), title='val_loss')
plt.gca().spines[['top', 'right']].set_visible(False)
plt.xlabel("Epochs")
plt.xticks(x_ticks)
plt.ylabel("Accuracy")

In [ ]:
from matplotlib import pyplot as plt
x_ticks = np.arange(0, 21 , step=1)
model_BCE_20epochs_df['train_acc'].plot(kind='line', figsize=(8, 4), title='train_acc')
plt.gca().spines[['top', 'right']].set_visible(False)
plt.xlabel("Epochs")
plt.xticks(x_ticks)
plt.ylabel("Accuracy")

In [ ]:
from matplotlib import pyplot as plt
model_BCE_20epochs_df['train_loss'].plot(kind='line', figsize=(8, 4), title='train_loss')
x_ticks = np.arange(0, 21 , step=1)
plt.gca().spines[['top', 'right']].set_visible(False)
plt.xlabel("Epochs")
plt.xticks(x_ticks)
plt.ylabel("Accuracy")

In [ ]:
# displaying the Dataframe
print('Dataframe:\n', model_BCE_20epochs_df)

In [ ]:
# saving the data as CSV file
#model_BCE_20epochs_df_csv = model_BCE_20epochs_df.to_csv('BCEmodel_20 epochs.csv', index=True) #insert in word

In [ ]:
import requests

# Download helper functions from Learn PyTorch repo (if not already downloaded)
if os.path.isfile("helper_function_script_at_given_path.py"):
  print("helper_functions.py already exists, skipping download")
else:
  print("Downloading helper_functions.py")
  os.chdir("given_path_for_helper_function_script")
  # Note: you need the "raw" GitHub URL for this to work
  request = requests.get("https://raw.githubusercontent.com/mrdbourke/pytorch-deep-learning/main/helper_functions.py")
  with open("helper_functions.py", "wb") as f:
    f.write(request.content)

In [ ]:
from helper_functions import accuracy_fn

In [ ]:
torch.manual_seed(42)
def eval_model(model: torch.nn.Module,
              data_loader: torch.utils.data.DataLoader,
              loss_fn: torch.nn.Module,
              accuracy_fn,
              ):
  """Returns a dictionary containing the results of model predicting on data_loader.

  Args:
      model (torch.nn.Module): A PyTorch model capable of making predictions on data_loader.
      data_loader (torch.utils.data.DataLoader): The target dataset to predict on.
      loss_fn (torch.nn.Module): The loss function of model.
      accuracy_fn: An accuracy function to compare the models predictions to the truth labels.

  Returns:
      (dict): Results of model making predictions on data_loader.
  """
  loss, acc = 0,0
  model.eval()
  with torch.inference_mode():
    for X,y in data_loader:
      X,y = X.to(device), y.to(device)
      # make predictions with the model
      y_pred = model(X)
      y_pred_label = torch.round(torch.sigmoid(y_pred))
      y=y.view(-1,1).float()

      # accumulate the loss and accuracy values per batch
      loss += loss_fn(y_pred, y)
      acc += accuracy_fn(y_true=y,
                         y_pred=y_pred_label)
      #print(f"accuracy: {acc}, loss: {loss}")
      # for accuracy, need the prediction labels (logits -> pred_prob -> pred_labels)
    loss /= len(data_loader)
    acc /= len(data_loader)

  return {"model_name": model.__class__.__name__,# only works when model was created with a class
          "model_loss": loss.item(),
          "model_acc": acc}
# calculate model 0 results on test dataset
model_BCE_20epochs_eval_results = eval_model(model=model_BCE_20epochs,
                                            data_loader=val_dataloader,
                                            loss_fn=loss_fn,
                                            accuracy_fn=accuracy_fn)
model_BCE_20epochs_eval_results

### 11. Confusion Matrix with BCEWithLogitsLoss

In [ ]:
#import tqdm for progress bar
from tqdm.auto import tqdm

# 1. make predictions with trained model
y_pred_label = []
y_true_label = []
model_BCE_20epochs.eval()
with torch.inference_mode():
  for X, y in tqdm(val_dataloader, desc='Making predictions...'):
    # send data and targets to target device
    X, y = X.to(device), y.to(device)
    # do the forward pass
    y_logit = model_BCE_20epochs(X)
    # turn predictions from logits -> prediction probabilities -> prediction labels
    y_pred = torch.round(torch.sigmoid(y_logit))
    # put predictions on CPU for evaluation
    y_pred_label.append(y_pred.cpu())
    # get true evaluation
    y_true_label.append(y.cpu()) # y is label

In [ ]:
# Concatenate list of predictions into a tensor
#print(y_pred_label)
#print(y_true_label)
y_true_tensor = torch.cat(y_true_label)
y_pred_tensor = torch.cat(y_pred_label).squeeze(dim=1)
y_pred_tensor.shape, y_true_tensor.shape


In [ ]:
# See if torchmetrics exists, if not, install it
try:
    import torchmetrics, mlxtend
    print(f"mlxtend version: {mlxtend.__version__}")
    assert int(mlxtend.__version__.split(".")[1]) >= 19, "mlxtend verison should be 0.19.0 or higher"
except:
    !pip install -q torchmetrics -U mlxtend # <- Note: If you're using Google Colab, this may require restarting the runtime
    import torchmetrics, mlxtend
    print(f"mlxtend version: {mlxtend.__version__}")

In [ ]:
# Import mlxtend upgraded version
import mlxtend
print(mlxtend.__version__)
assert int(mlxtend.__version__.split(".")[1]) >= 19 # should be version 0.19.0 or higher

In [ ]:
class_names #y_pred_label

In [ ]:
from torchmetrics import ConfusionMatrix
from mlxtend.plotting import plot_confusion_matrix

# 2. Setup confusion matrix instance and compare predictions to targets
confmat = ConfusionMatrix(num_classes=2, task='BINARY')
confmat_tensor = confmat(preds = y_pred_tensor,
                         target = y_true_tensor)

# plot the confusion matrix
fig, ax = plot_confusion_matrix(
    conf_mat = confmat_tensor.numpy(), # matplotlib likes working with NumPy
    class_names = class_names, # turn the row and column labels into class names
    figsize=(5,5),
    cmap=plt.get_cmap('PiYG')
);

In [ ]:
loaded_model_BCE_20epochs.to(device)

### 12. Test Set EVALUATE
Now to evaluate our saved and loaded model on test_dataloader, let's perform inference with it (make predictions) on the test dataset.

In [ ]:
# Evaluate the test set
torch.manual_seed(42)

loaded_model_BCE_20epochs_results = eval_model(
    model=loaded_model_BCE_20epochs,
    data_loader=test_dataloader,
    loss_fn=loss_fn,
    accuracy_fn=accuracy_fn
)
loaded_model_BCE_20epochs_results